# Testing `run_symmetrized_readout_new`

In [1]:
from math import pi
from typing import List
import itertools
import numpy as np
import networkx as nx

from pyquil import Program, get_qc
from pyquil.api import QuantumComputer
from pyquil.gates import *
from forest.benchmarking.utils import bitstring_prep

**get a quantum computers, because we need one**

In [2]:
from forest.benchmarking.compilation import basic_compile
from pyquil.api._qac import AbstractCompiler

# we need to do this otherwise everything gets complied away
class DummyCompiler(AbstractCompiler):
    def get_version_info(self):
        return {}

    def quil_to_native_quil(self, program: Program):
        return basic_compile(program)

    def native_quil_to_executable(self, nq_program: Program):
        return nq_program

qc = get_qc("8q-qvm")
qc.compiler = DummyCompiler()

**define some two qubit asymmetric noise operators**

In [3]:
def two_qubit_bit_flip_operators(p00,p01,p10,p11):
    """
    Return a special case of a two qubit asymmetric bit flip kraus operators.
    
    Suppose we prepare a two qubit state |i,j> = |i>\otimes|j> for i,j \in {0,1}.
    
    Then pij := Pr(measured=ij|prepared=ij). So if pij = 1 no flip happens.
    
    For example consider p00 = 1-epsilon then a flip happens with probablity epsilon.
    The flip is symmetrically superposed over flipping to the states |0,1>, |1,0>, and |1,1>.
    The asymmetry comes from the fact that p00 does not have to be equal to p10 etc.
    
    :param p00: the probablity of |0,0> to remain in |0,0>
    :param p01: the probablity of |0,1> to remain in |0,1>
    :param p10: the probablity of |1,0> to remain in |1,0>
    :param p11: the probablity of |1,1> to remain in |1,1>
    :returns: a list of four Kraus operators.
    """
    p00e = 1.0 - p00
    p01e = 1.0 - p01
    p10e = 1.0 - p10
    p11e = 1.0 - p11
    kI = np.array([[np.sqrt(1-p00e), 0.0, 0.0, 0.0], [0.0, np.sqrt(1-p01e), 0.0, 0.0], [0.0, 0.0, np.sqrt(1-p10e), 0.0], [0.0, 0.0, 0.0, np.sqrt(1-p11e)]])
    k00 = np.sqrt(p00e/3) * ( _flip_matrix(0,1) + _flip_matrix(0,2) + _flip_matrix(0,3) )
    k01 = np.sqrt(p01e/3) * ( _flip_matrix(1,0) + _flip_matrix(1,2) + _flip_matrix(1,3) )
    k10 = np.sqrt(p10e/3) * ( _flip_matrix(2,0) + _flip_matrix(2,1) + _flip_matrix(2,3) )
    k11 = np.sqrt(p11e/3) * ( _flip_matrix(3,0) + _flip_matrix(3,1) + _flip_matrix(3,2) )
    return kI, k00, k01, k10, k11

def _flip_matrix(i,j,dim=4):
    mat = np.zeros((dim,dim))
    #mat.itemset((i,j),1)
    mat.itemset((j,i),1)
    return mat

def append_kraus_to_gate(kraus_ops, g):
    """
    Follow a gate `g` by a Kraus map described by `kraus_ops`.

    :param list kraus_ops: The Kraus operators.
    :param numpy.ndarray g: The unitary gate.
    :return: A list of transformed Kraus operators.
    """
    return [kj.dot(g) for kj in kraus_ops]

In [4]:
nbits = 2
prepnames = []
prepprogs = []
binarray = []
for flip_array in itertools.product([0, 1], repeat=nbits):
    prepprogs.append(bitstring_prep(list(range(0,nbits)),flip_array))
    prepnames.append(''.join(str(x) for x in flip_array))
    binarray.append(list(flip_array))

In [5]:
from pyquil.quil import DefGate

II_mat = np.eye(4)
II_definition = DefGate("II", II_mat)
II = II_definition.get_constructor()
kraus_ops = two_qubit_bit_flip_operators(0.1,1,1,1)

p = Program(II_definition)

p.define_noisy_gate("II", [0, 1],  append_kraus_to_gate(kraus_ops, II_mat))
p += Program(II(0,1))


In [6]:
progRAM = p.copy()

prog = p.copy()
ro = prog.declare('ro', 'BIT', 2)
prog += MEASURE(0, ro[0])
prog += MEASURE(1, ro[1])
prog.wrap_in_numshots_loop(10)

# `run_and_measure` 

the noise is only on qubits 0 and 1

In [7]:
qc.run_and_measure(progRAM,trials=10)

{0: array([1, 1, 1, 0, 1, 0, 1, 0, 1, 0]),
 1: array([1, 1, 0, 0, 1, 1, 0, 1, 1, 0]),
 2: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 3: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 4: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 5: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 6: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 7: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

# `run` the noise models works :)

In [8]:
qc.run(prog)

array([[0, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [1, 0],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [1, 1]])

# `run_symmetrized_readout_new` also works if you turn the compiler off

In [12]:
qc.run_symmetrized_readout_new(progRAM, trials=10)

array([[0, 1],
       [1, 1],
       [0, 1],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 0],
       [0, 0]])

# check the confusion matrix

In [13]:
import pandas as pd

In [14]:
nbits = 2
prepnames = []
prepprogs = []
binarray = []
for flip_array in itertools.product([0, 1], repeat=nbits):
    prepprogs.append(bitstring_prep(list(range(0,nbits)),flip_array))
    prepnames.append(''.join(str(x) for x in flip_array))
    binarray.append(list(flip_array))

In [15]:
# kraus_ops = two_qubit_bit_flip_operators(1,0.7,1,1)

# # add noise
# for prog in prepprogs:
#     prog.inst(II_definition)
#     prog.define_noisy_gate("II", [0, 1],  append_kraus_to_gate(kraus_ops, II_mat))
#     prog.inst(II(0,1))

Note there is a really big hack here... I had to hard code the noise in.

the problem is that if you add the symmetrization using this method it happens after the noise which wont do anything

**trival / no symm**

In [17]:
data = []
num_shots = 6000

for prepname,program in zip(prepnames,prepprogs):
    answer = qc.run_symmetrized_readout_new(program, trials=num_shots, symm_type=0)
    counts=[]
    for cdx in binarray:
        counts.append(answer.tolist().count(cdx)/num_shots)

    dicty = dict(zip(['Prep']+[ 'Meas '+name for name in prepnames], [prepname]+counts)) 
    data.append(dicty)

df = pd.DataFrame(data)
ndf = df.set_index('Prep',drop=True)
ndf

,Meas 00,Meas 01,Meas 10,Meas 11
Prep,,,,
00,0.705,0.093833,0.104333,0.096833
01,0.000,1.000000,0.000000,0.000000
10,0.000,0.000000,1.000000,0.000000
11,0.000,0.000000,0.000000,1.000000


**One design**

In this case does not help much

In [18]:
data = []
num_shots = 6000

for prepname,program in zip(prepnames,prepprogs):
    answer = qc.run_symmetrized_readout_new(program, trials=num_shots, symm_type=1)
    counts=[]
    for cdx in binarray:
        counts.append(answer.tolist().count(cdx)/num_shots)

    dicty = dict(zip(['Prep']+[ 'Meas '+name for name in prepnames], [prepname]+counts)) 
    data.append(dicty)

df = pd.DataFrame(data)
ndf = df.set_index('Prep',drop=True)
ndf

,Meas 00,Meas 01,Meas 10,Meas 11
Prep,,,,
00,0.853000,0.049000,0.048167,0.049833
01,0.000000,1.000000,0.000000,0.000000
10,0.000000,0.000000,1.000000,0.000000
11,0.049333,0.048833,0.049500,0.852333


**two design**

:) it is working!!

In [19]:
data = []
num_shots = 6000

for prepname,program in zip(prepnames,prepprogs):
    answer = qc.run_symmetrized_readout_new(program, trials=num_shots, symm_type=2)
    counts=[]
    for cdx in binarray:
        counts.append(answer.tolist().count(cdx)/num_shots)

    dicty = dict(zip(['Prep']+[ 'Meas '+name for name in prepnames], [prepname]+counts)) 
    data.append(dicty)

df = pd.DataFrame(data)
ndf = df.set_index('Prep',drop=True)
ndf

,Meas 00,Meas 01,Meas 10,Meas 11
Prep,,,,
00,0.918333,0.027333,0.027000,0.027333
01,0.026167,0.923333,0.024667,0.025833
10,0.024500,0.025667,0.924333,0.025500
11,0.024500,0.025333,0.026167,0.924000


**three design**

of course it works here

In [20]:
data = []
num_shots = 6000

for prepname,program in zip(prepnames,prepprogs):
    answer = qc.run_symmetrized_readout_new(program, trials=num_shots, symm_type=3)
    counts=[]
    for cdx in binarray:
        counts.append(answer.tolist().count(cdx)/num_shots)

    dicty = dict(zip(['Prep']+[ 'Meas '+name for name in prepnames], [prepname]+counts)) 
    data.append(dicty)

df = pd.DataFrame(data)
ndf = df.set_index('Prep',drop=True)
ndf

,Meas 00,Meas 01,Meas 10,Meas 11
Prep,,,,
00,0.925667,0.027000,0.022833,0.024500
01,0.027500,0.922500,0.024333,0.025667
10,0.027167,0.023167,0.922833,0.026833
11,0.027000,0.025833,0.025000,0.922167


In [45]:
#from pyquil.api._quantum_computer import _flip_array_to_prog, symmetrization

In [46]:
#progy = _flip_array_to_prog((1,0,1,0,1,1),[0,1,2,3,4,5])

In [47]:
#progy.out().splitlines()

In [48]:
#prog, flip_array = symmetrization(Program(I(0),I(1)), [0,1], 2) 

In [49]:
#flip_array

In [50]:
#print(prog[0].out().splitlines())